<a href="https://colab.research.google.com/github/Sritam33/Deep_learning/blob/main/Titanic_Multilayer_Perceptron%2BDropout_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [ ]:
# load the titanic dataset
train_data = pd.read_csv("/content/train (1).csv")
test_data = pd.read_csv("/content/test (1).csv")

In [ ]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
# data preprocesing
def preprocess_data(data):
  # fill missing values
  data["Age"].fillna(data["Age"].mean(),inplace = True)
  data["Embarked"].fillna(data["Embarked"].mode()[0],inplace = True)
  data["Fare"].fillna(data["Fare"].mean(),inplace = True)
  #data["Cabin"].fillna(data["Cabin"].mode()[0],inplace = True)
  # encode categorical features
  label_encoder = LabelEncoder()
  data["Sex"] = label_encoder.fit_transform(data["Sex"])
  data["Embarked"] = label_encoder.fit_transform(data["Embarked"])
  # select features
  features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
  X = data[features].values
  y = data["Survived"].values
  return X,y

In [ ]:
# preprocess the training data
X_train, y_train = preprocess_data(train_data)
# split the training data into training and validation sets
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.2,random_state = 42)
# convert data to pytorch tensors
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor= torch.FloatTensor(X_val)
y_train_tensor = torch.LongTensor(y_train)
y_val_tensor = torch.LongTensor(y_val)

In [ ]:
train_dataset = TensorDataset(X_train_tensor,y_train_tensor)
val_dataset = TensorDataset(X_val_tensor,y_val_tensor)
batch_size = 64
train_dataloader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(val_dataset,batch_size = batch_size)

In [ ]:
# define the neural network
class TitanicNN(nn.Module):
  def __init__(self,input_size):
    super(TitanicNN,self).__init__()
    self.fc1 = nn.Linear(input_size,64)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(64,2) # Output layer with 2 classes (Survived or Not Survived)
    self.softmax = nn.Softmax(dim=1)

  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [ ]:
# instantiate the model
input_size = X_train.shape[1]
model = TitanicNN(input_size)
X_train.shape

(712, 7)

In [ ]:
input_size

7

In [ ]:
# loss fuction and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
# training the model
epochs = 10
for epoch in range(epochs):
  correct = 0
  total = 0
  model.train()
  for batch in train_dataloader:
    X_batch, y_batch = batch
    optimizer.zero_grad()
    outputs = model(X_batch)
    loss = loss_fn(outputs, y_batch)
    loss.backward()
    optimizer.step()

    # evaluate the model on the validation set
  model.eval()

  with torch.no_grad():
    for batch in val_dataloader:
       X_batch, y_batch = batch
       outputs = model(X_batch)
       _,predicted = torch.max(outputs.data,1)
       correct += (predicted == y_batch).sum().item()
       total += y_batch.size(0)
       accuracy = correct / total
  print(f"validation accuracy:{accuracy * 100 :.2f}%")

validation accuracy:70.39%
validation accuracy:70.95%
validation accuracy:70.95%
validation accuracy:70.95%
validation accuracy:70.95%
validation accuracy:70.95%
validation accuracy:71.51%
validation accuracy:71.51%
validation accuracy:69.27%
validation accuracy:67.60%
